In [1]:
! pip install pandas scikit-learn skl2onnx tensorflow tf2onnx

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [3]:
! ls

data                     output                   recognizerServer.ipynb
info.txt                 positions                recognizerTraining.ipynb


### 60hz

In [ ]:
# Folder with CSV files – each file contains one gesture of 30 frames
data_folder = os.path.join(os.getcwd(), 'newData')
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))
new_model = os.path.join(os.getcwd(), 'newModel')

flattened_samples = []
labels = []

for f in csv_files:
    df = pd.read_csv(f)
    # Optionally, sort by frame if necessary:
    df = df.sort_values(by='Frame')
    # Map hand strings to numbers if needed:
    df['Hand'] = df['Hand'].map({'left': 1, 'right': 0})
    # Confirm that all rows in file belong to same gesture:
    gesture_label = df['Label'].iloc[0]
    # Remove non-feature columns if desired (keep if they are features too)
    # Here we remove 'Frame' and 'Label'
    feature_df = df.drop(columns=['Frame', 'Label'])
    # Flatten the 2D array (shape 30 x n_features) into 1D vector
    flattened = feature_df.to_numpy().flatten()  
    flattened_samples.append(flattened)
    labels.append(gesture_label)

# Create a new DataFrame – columns can be auto-named or you can specify them explicitly.
# For example, the feature dimension is 30 * (number of feature columns in one file)
num_features = flattened_samples[0].shape[0]
columns = [f"f{i}" for i in range(num_features)]
new_df = pd.DataFrame(flattened_samples, columns=columns)
new_df['Label'] = labels

# Optionally, save the reshaped data to a new CSV file
# output_csv = os.path.join(data_folder, "reshaped_gesture_data_30.csv")
# new_df.to_csv(output_csv, index=False)
# print(f"Reshaped data saved to {output_csv}")

### 30hz

In [10]:
# Folder with CSV files – each file contains one gesture of 30 frames (60Hz recordings)
data_folder = os.path.join(os.getcwd(), 'newData')
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))
new_model = os.path.join(os.getcwd(), 'newModel')

samples_even = []
samples_odd = []
labels = []

for f in csv_files:
    df = pd.read_csv(f)
    # Optionally, sort by frame if necessary:
    df = df.sort_values(by='Frame')
    # Map hand strings to numbers if needed:
    df['Hand'] = df['Hand'].map({'left': 1, 'right': 0})
    
    # Confirm that all rows in file belong to the same gesture and get its label
    gesture_label = df['Label'].iloc[0]
    # Remove non-feature columns ('Frame' and 'Label'); keep if you want to use 'Hand' as a feature
    feature_df = df.drop(columns=['Frame', 'Label'])
    
    # Split into even and odd frames:
    even_df = feature_df.iloc[::2]  # frames 0,2,4,...,28 (15 frames)
    odd_df  = feature_df.iloc[1::2]  # frames 1,3,5,...,29 (15 frames)
    
    # Flatten each subset into a 1D vector
    flattened_even = even_df.to_numpy().flatten()
    flattened_odd  = odd_df.to_numpy().flatten()
    
    samples_even.append(flattened_even)
    samples_odd.append(flattened_odd)
    labels.append(gesture_label)  # same label for both splits

# Create DataFrames for even and odd samples. They should have the same number of features.
num_features = samples_even[0].shape[0]
columns = [f"f{i}" for i in range(num_features)]

df_even = pd.DataFrame(samples_even, columns=columns)
df_even['Label'] = labels
#df_even['SampleType'] = 'even'   # Optional field to know sample origin

df_odd = pd.DataFrame(samples_odd, columns=columns)
df_odd['Label'] = labels
#df_odd['SampleType'] = 'odd'

# Combine both to get a 15Hz dataset with doubled samples
df_15hz = pd.concat([df_even, df_odd], ignore_index=True)

# Optionally, save the new DataFrame to a CSV file
# output_csv = os.path.join(data_folder, "reshaped_gesture_data_15hz.csv")
# df_15hz.to_csv(output_csv, index=False)
# print(f"Saved 15Hz dataset with even/odd split to: {output_csv}")

### training

In [11]:
# data_df = new_df
data_df = df_15hz

# Define feature columns and target. Here we use all columns except 'Label' as features.
feature_cols = [col for col in data_df.columns if col not in ['Label']]
X = data_df[feature_cols].values
y = data_df['Label'].values

# Split data: 60% training, 20% validation, 20% test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)
# Note: 0.25 * 0.8 = 0.2, so overall 60% train, 20% validation, 20% test

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Instantiate the SVM model
svm_model = SVC(kernel='rbf', probability=True, random_state=42)

# Perform 5-fold cross-validation on the training set (using StratifiedKFold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
cv_scores = cross_val_score(svm_model, X_train_scaled, y_train, cv=cv)
print("5-Fold Cross-Validation scores on training set:", cv_scores)
print("Mean CV score:", np.mean(cv_scores))

# Train the SVM classifier using the full training set
svm_model.fit(X_train_scaled, y_train)

# Validate the model
y_val_pred = svm_model.predict(X_val_scaled)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Test the model
y_test_pred = svm_model.predict(X_test_scaled)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Save the model and scaler for later use
joblib.dump(svm_model, os.path.join(new_model, "svm_model.pkl"))
joblib.dump(scaler, os.path.join(new_model, "scaler.pkl"))

5-Fold Cross-Validation scores on training set: [0.94339623 0.94339623 0.8490566  0.83018868 0.80769231]
Mean CV score: 0.8747460087082729
Validation Accuracy: 0.8863636363636364
Validation Classification Report:
              precision    recall  f1-score   support

     destroy       0.89      1.00      0.94        16
        idle       0.88      0.88      0.88        40
        pick       0.85      0.69      0.76        16
       prune       0.94      1.00      0.97        16

    accuracy                           0.89        88
   macro avg       0.89      0.89      0.89        88
weighted avg       0.88      0.89      0.88        88

Test Accuracy: 0.9318181818181818
Test Classification Report:
              precision    recall  f1-score   support

     destroy       0.94      1.00      0.97        16
        idle       0.90      0.95      0.93        40
        pick       1.00      0.75      0.86        16
       prune       0.94      1.00      0.97        16

    accuracy      

['/Users/michelelamon/Documents/UNITN/AdvancedHCI/xrnn/python/gestures/newModel/scaler.pkl']

## CNN experiments 

In [7]:
# import os
# import glob
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping

# # Directory with your CSV files
# data_folder = os.path.join(os.getcwd(), 'data')
# csv_files = glob.glob(os.path.join(data_folder, "*.csv"))

# # Read and combine all CSV files
# df_list = [pd.read_csv(f) for f in csv_files]
# data_df = pd.concat(df_list, ignore_index=True)

# # Map hand strings to numbers (if you want to retain it)
# data_df['Hand'] = data_df['Hand'].map({'left': 1, 'right': 0})

# # We'll use all columns EXCEPT these non-feature ones
# non_feature = ['Label']
# feature_cols = [col for col in data_df.columns if col not in non_feature]
# X = data_df[feature_cols].values
# y = data_df['Label'].values

# # Encode labels to integers and count classes
# le = LabelEncoder()
# y_encoded = le.fit_transform(y)
# num_classes = len(le.classes_)

# # Split data
# X_train_val, X_test, y_train_val, y_test = train_test_split(
#     X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)
# # Overall: 60% train, 20% validation, 20% test

# # Normalize features (fit only on training data)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

# # Our features count is 81 (84 columns - Frame, Hand, Label)
# num_features = X_train.shape[1]  # should be 81

# # Reshape for Conv1D: (samples, steps, channels)
# X_train = X_train.reshape(-1, num_features, 1)
# X_val = X_val.reshape(-1, num_features, 1)
# X_test = X_test.reshape(-1, num_features, 1)

# # Build a small 1D CNN model
# model = Sequential([
#     Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
#     # Second convolution to learn higher-level combinations; depth=2
#     Conv1D(filters=32, kernel_size=3, activation='relu'),
#     GlobalAveragePooling1D(),
#     Dense(16, activation='relu'),
#     Dropout(0.2),
#     Dense(num_classes, activation='softmax')
# ])

# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model.summary()

# # Early stopping to help avoid overfitting
# es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Train the model
# history = model.fit(X_train, y_train, epochs=50, batch_size=16,
#                     validation_data=(X_val, y_val), callbacks=[es])

# # Evaluate the model on the test set
# loss, accuracy = model.evaluate(X_test, y_test)
# print("Test accuracy: {:.2f}%".format(accuracy * 100))

# # Save the trained model if needed
# model.save(os.path.join(data_folder, "small_cnn_model.h5"))

In [8]:
# import os
# import tensorflow as tf
# import tf2onnx

# # Define paths
# data_folder = os.path.join(os.getcwd(), "data")
# h5_model_path = os.path.join(data_folder, "small_cnn_model.h5")
# onnx_model_path = os.path.join(data_folder, "small_cnn_model.onnx")

# # Load the trained Keras model
# model = tf.keras.models.load_model(h5_model_path)

# # Manually set output_names if not present
# if not hasattr(model, "output_names"):
#     model.output_names = [output.name for output in model.outputs]

# # Define the input signature based on your model's input shape: (None, num_features, 1)
# input_spec = (tf.TensorSpec((None, model.input_shape[1], model.input_shape[2]), tf.float32, name="input"),)

# # Convert the model to ONNX format using tf2onnx
# model_proto, external_tensor_storage = tf2onnx.convert.from_keras(
#     model,
#     input_signature=input_spec,
#     opset=13,
#     output_path=onnx_model_path
# )

# print("ONNX model exported to:", onnx_model_path)

In [9]:
# import os
# import glob
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, LabelEncoder
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping
# import tf2onnx

# # ------------------------
# # Data Preparation Section
# # ------------------------
# data_folder = os.path.join(os.getcwd(), "data")
# csv_files = glob.glob(os.path.join(data_folder, "*.csv"))

# # Read and combine CSV files
# df_list = [pd.read_csv(f) for f in csv_files]
# data_df = pd.concat(df_list, ignore_index=True)

# # Map hand strings to numbers (if needed)
# data_df['Hand'] = data_df['Hand'].map({'left': 1, 'right': 0})

# # Use all columns except non-feature ones: Frame, Hand, and Label
# non_feature = ['Label']
# feature_cols = [col for col in data_df.columns if col not in non_feature]
# X = data_df[feature_cols].values
# y = data_df['Label'].values

# # Encode labels to integers
# le = LabelEncoder()
# y_encoded = le.fit_transform(y)
# num_classes = len(le.classes_)

# # Split data: 60% train, 20% validation, 20% test
# X_train_val, X_test, y_train_val, y_test = train_test_split(
#     X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)

# # Normalize features on training set
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

# # Determine number of features (for example, 81 for 84 columns minus 3 non-features)
# num_features = X_train.shape[1]

# # Reshape data for Conv1D: (samples, steps, channels)
# X_train = X_train.reshape(-1, num_features, 1)
# X_val = X_val.reshape(-1, num_features, 1)
# X_test = X_test.reshape(-1, num_features, 1)

# # ------------------------------------
# # Build and Train a Smaller CNN Model
# # ------------------------------------
# model = Sequential([
#     # Single Conv1D layer with fewer filters
#     Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
#     # Global pooling reduces parameters while still summarizing features
#     GlobalAveragePooling1D(),
#     # A small dense layer
#     Dense(8, activation='relu'),
#     Dropout(0.2),
#     Dense(num_classes, activation='softmax')
# ])

# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model.summary()

# # Early stopping to help avoid overfitting
# es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Train the model
# history = model.fit(X_train, y_train, epochs=50, batch_size=16,
#                     validation_data=(X_val, y_val), callbacks=[es])

# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)
# print("Test accuracy: {:.2f}%".format(accuracy * 100))

# # Save the Keras model to H5 format
# h5_model_path = os.path.join(data_folder, "small_cnn_model.h5")
# model.save(h5_model_path)
# print("Keras model saved to:", h5_model_path)

# # ------------------------------
# # Export the Model to ONNX Format
# # ------------------------------
# onnx_model_path = os.path.join(data_folder, "small_cnn_model.onnx")

# # Load the saved Keras model (for conversion clarity)
# model = tf.keras.models.load_model(h5_model_path)

# # Manually set output names if missing
# if not hasattr(model, "output_names"):
#     model.output_names = [output.name for output in model.outputs]

# # Define the input signature based on the model's input shape: (None, num_features, 1)
# input_spec = (tf.TensorSpec((None, model.input_shape[1], model.input_shape[2]), tf.float32, name="input"),)

# # Convert the Keras model to ONNX using tf2onnx
# model_proto, external_tensor_storage = tf2onnx.convert.from_keras(
#     model,
#     input_signature=input_spec,
#     opset=13,
#     output_path=onnx_model_path
# )

# print("ONNX model exported to:", onnx_model_path)